# L1.3 Regresión lineal múltiple

En esta lectura generaremos un modelo de regresión lineal múltiple, revisaremos si el modelo es útil (si alguna variable está asociada significativamente a la respuesta), analizaremos una variable de forma específica, y revisaremos la calidad del ajuste del modelo, tanto en datos de entrenamiento, como en datos de validación.

Por favor no modifiques las celdas con las instrucciones, y solamente escribe código en las celdas donde así se te indica. **Si en algún momento seleccionas por error una celda de instrucciones y su apariencia cambia, simplemente presiona "Ctrl + Enter".**

Dentro de las celdas de código, las líneas que inician con un "\#" son comentarios y no se ejecutarán, simplemente sirven como instrucciones o descripciones útiles para ustedes.

En este caso trabajaremos con los datos del Wine Quality Data Set, generados por Paulo Cortez y un grupo de investigación de la University of Minho, en Portugal, disponibles en [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/186/wine+quality). Los investigadores utilizaron estos datos para tratar de modelar las preferencias de consumo de vino, generando un índice de calidad. Nosotros trataremos de usar la información disponible, justamente para predecir dicho índice. Puedes encontrar la base de datos junto con el material del curso; el archivo lleva por nombre "L1.3 Vino Tinto.csv".

En la primera celda cargaremos la base de datos en una variable de nombre `df` usando la función `read_csv`de la librería de pandas, revisaremos el tamaño de la base de datos con la función `shape` y revisaremos los nombres de las variables con el operador `columns`. No olvides asegurarte de que el archivo a leer esté en el mismo directorio donde se encuentra esta libreta.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import pandas as pd<br>
    df = pd.read_csv("Vino Tinto.csv")<br>
    print(df.shape)<br>
    print(df.columns)
</details>

In [2]:
# Importa la librería pandas
import pandas as pd
# Lee el archivo
df = pd.read_csv('L1.3 Vino Tinto.csv')
# Imprime en consola las dimensiones de los datos
print(df.shape)
# Imprime en consola los nombres de las variables
print(df.columns)

(1599, 12)
Index(['acidezFija', 'acidezVolatil', 'acidoCitrico', 'azucarResidual',
       'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'densidad',
       'pH', 'sulfatos', 'alcohol', 'calidad'],
      dtype='object')


Como podrás notar, nuestra base de datos cuenta con 1599 observaciones distintas y 12 variables. Ya que una de esas variables, la calidad, será el valor a predecir, el modelo trabajará con 11 variables distintas: acidez fija, acidez volátil, ácido cítrico, azúcar residual, cloruros, dióxido de azufre libre, dióxido de azufre total, densidad, pH, sulfatos, y alcohol.

Ya que nos interesa obtener al final del análisis tanto un error de entrenamiento, como uno de validación, separemos nuestra base de datos en dos, justamente en un subset de entrenamiento y uno de prueba. Primero realicemos el proceso por nuestra cuenta, de manera que entendamos lo que le sucede a los datos.

El primer paso consistirá en usar el operador `sample` de pandas, para generar una muestra aleatoria de elementos de un objeto; especificaremos como parámetro que queremos obtener una fracción del 80% de los elementos originales. Almacenaremos el resultado de esta operación en una variable de nombre `train`.

Posteriormente, generaremos una variable de nombre `test` donde almacenaremos los elementos que no fueron seleccionados para la etapa de entrenamiento. Para lograrlo, usaremos el operador `drop`, también de pandas, que se usa para eliminar ciertas filas (o columnas, dependiendo de la necesidad) de un *data frame*. Es decir, copiaremos la base de datos original, con excepción de las filas utilizadas para el subset de entrenamiento. Para determinar exactamente qué filas eliminar, usaremos el operador `index`, que entrega un identificador específico a cada observación.

Para comprobar que el proceso fue exitoso, imprimiremos en consola las dimensiones de los nuevos subsets, así como las primeras 5 filas del set de entrenamiento.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    train = df.sample(frac = 0.8)<br>
    test = df.drop(train.index)<br>
    print("Train:", train.shape)<br>
    print("Test:",test.shape)<br>
    print(train.head())
</details>


In [3]:
# Genera datos de entrenamiento
train = df.sample(frac=0.8)
# Genera datos de validación
test = df.drop(train.index)
# Imprime dimensiones de datos de entrenamiento
print("Train:", train.shape)
# Imprime dimensiones de datos de prueba
print("Test:", test.shape)
# Imprime primeras 5 filas de datos de entrenamiento
print(train.head())

Train: (1279, 12)
Test: (320, 12)
      acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
878          8.8           0.61          0.19             4.0     0.094   
1323         9.1           0.36          0.39             1.8     0.060   
1286         7.1           0.16          0.44             2.5     0.068   
1068        11.1           0.31          0.53             2.2     0.060   
529          9.9           0.63          0.24             2.4     0.077   

      dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  \
878                 30.0                69.0   0.99787  3.22      0.50   
1323                21.0                55.0   0.99495  3.18      0.82   
1286                17.0                31.0   0.99328  3.35      0.54   
1068                 3.0                10.0   0.99572  3.02      0.83   
529                  6.0                33.0   0.99740  3.09      0.57   

      alcohol  calidad  
878      10.0        6  
1323     11.0       

Ahora sabemos que contamos con 1279 observaciones de entrenamiento y 320 de validación. Es común que los sets de validación sean mucho más pequeños que los de entrenamiento, pues entre mayor información utilicemos para entrenar, un mejor dempeseño tenderá a tener el modelo.

Otra forma de lograr el mismo objetivo, es usando la función `train_test_split` de la librería sklearn.model_selection. En la siguiente celda repetiremos el proceso previo usando esta función, definiendo los nombres de los datasets de salida en la misma línea, y dando como argumentos el nombre del *data frame* original y el tamaño (proporción entre 0 y 1) del set de entrenamiento que deseamos obtener.

Igualmente, validaremos que el proceso haya sido exitoso imprimiendo en consola las dimensiones de los nuevos subsets, y revisando las primeras 5 filas del subset de entrenamiento.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    from sklearn.model_selection import train_test_split<br>
    train, test = train_test_split(df, train_size = 0.8)<br>
    print("Train:", train.shape)<br>
    print("Test:",test.shape)<br>
    print(train.head())
</details>

In [4]:
# Importa la función train_test_split
from sklearn.model_selection import train_test_split
# Genera los datos de entrenamiento y validación
train, test = train_test_split(df, train_size=0.8)
# Imprime en consola las dimensiones de los datos de entrenamiento
print("Train:", train.shape)
# Imprime en consola las dimensiones de los datos de prueba
print("Test:", test.shape)
# Imprime en consola las primeras 5 filas de los datos de entrenamiento
print(train.head())


Train: (1279, 12)
Test: (320, 12)
      acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
107          6.2          0.630          0.31             1.7     0.088   
776          6.9          0.765          0.18             2.4     0.243   
665          9.4          0.590          0.14             2.0     0.084   
1589         6.6          0.725          0.20             7.8     0.073   
385          7.4          0.630          0.07             2.4     0.090   

      dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  \
107                 15.0                64.0   0.99690  3.46      0.79   
776                  5.5                48.0   0.99612  3.40      0.60   
665                 25.0                48.0   0.99810  3.14      0.56   
1589                29.0                79.0   0.99770  3.29      0.54   
385                 11.0                37.0   0.99790  3.43      0.76   

      alcohol  calidad  
107       9.3        5  
776      10.3       

Como podrás observar, obtuvimos los mismos resultados en términos del tamaño de los objetos. Pero, las observaciones  no son las mismas entre ambas metodologías. Esto no se debe a que el proceso sea distinto, sino que en ambos casos partimos de una selección aleatoria de sujetos. De hecho, si vuelven a correr las celdas anteriores varias veces, cada ocasión observarán un resultado distinto.

Es momento de realizar la regresión lineal múltiple. En esta ocasión nos apoyaremos de la misma función que utilizamos en la lectura interactiva del Módulo 3, `OLS` de la librería statsmodels.api. Primero definiremos $X$ y $Y$ a partir de los datos de entrenamiento; para generar el primer elemento simplemente eliminaremos la columna `calidad` del dataset `train` usando el operador `drop`; para el segundo elemento simplemente tomaremos dicha columna. Después definieremos el modelo `model` con la función `OLS`, utilizando como parámetros `Y` y `sm.add_constant(X)` para asegurarnos que el modelo incluya $\beta_0$. Finalmente, estimaremos el valor de cada coeficiente con el método `fit`, almacenando los resultados en `results`. Para revisar los resultados obtenido, imprimiremos en pantalla un resumen utilizando el método `summary()`.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import statsmodels.api as sm<br>
    X = train.drop('calidad', axis = 1)<br>
    Y = train.calidad<br>
    model = sm.OLS(Y,sm.add_constant(X))<br>
    results = model.fit()<br>
    print(results.summary())
</details>

In [5]:
# Importar librería
import statsmodels.api as sm
# Generar elemento X
X = train.drop('calidad', axis=1)
# Generar elemento Y
Y = train['calidad']
# Definir el tipo de modelo
model = sm.OLS(Y, sm.add_constant(X))
# Ajustar el modelo para obtener resultados
results = model.fit()
# Imprimir un resumen de los resultados
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                calidad   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     68.39
Date:                Wed, 20 Aug 2025   Prob (F-statistic):          5.48e-120
Time:                        19:36:26   Log-Likelihood:                -1252.4
No. Observations:                1279   AIC:                             2529.
Df Residuals:                    1267   BIC:                             2591.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 17.6402     23

En el resumen de resultados previo podemos revisar múltiples detalles. Por ejemplo: el valor de cada uno de los coeficientes del modelo, el *p-value* asociado a cada uno de dichos coeficientes bajo una prueba de significancia de coeficientes individuales, el intervalo de confianza para cada coeficiente, el estadístico *F* y su *p-value* asociado, el valor de la $R^2$, etc. Sin embargo, por única ocasión, tratemos de comprobar si lo aprendido previamente nos ayuda a calcular por nuestra cuenta algunos de estos valores.

Antes de hacerlo, revisa con atención las características del modelo. El *p-value* asociado al estadístico *F* nos indica que sí hay al menos una variable asociada de forma significativa con la salida. Adicionalmente, los *p-values* de los coeficientes individuales nos muestran qué variables específicamente son de interés (acidez volátil, cloruros, y alcohol, por ejemplo).

Primero calcularemos el estadístico *F*, tal y como se describió en clase. Empezaremos calculando el EMS y el RMS a partir del ESS y del RSS; recuerden que el ESS se define como la sumatoria de la diferencia cuadrada entre el valor estimado de $Y$ y el valor promedio de $Y$, mientras que el RSS se define como la sumatoria de la diferencia cuadrada entre el valor actual de $Y$ y el valor estimado de $Y$.

Realizarems este proceso en múltiples pasos:
1. Estimar el valor de $Y$ a partir de los coeficientes estimados del modelo, usando el método `predict` sobre los datos `X`, tras agregarles una columna de unos con la función `add_constant`. En sí, esta función simplemente realiza el producto punto entre los datos y los coeficientes del modelo.
2. Calcular el valor promedio de $Y$, usando la función `mean` de numpy.
3. Calcular ESS.
4. Calcular $m$ como la cantidad de variables de los datos de entrenamiento, usando la función `shape` y tomando solo el segundo elemento `[1]`.
5. Calcular EMS (ESS/m).
6. Calcular RSS.
7. Calcular $n$ como la cantidad de observaciones de los datos de entrenamiento, usando la función `shape` y tomando solo el primero elemento `[0]`.
8. Calcular RMS (RSS/(n-m-1)).
9. Calcular F (EMS/RMS).
10. Calcular el *p-value* asociado a dicho estadístico *F*. Para lograrlo, usaremos la función `f.sf` de la librería scipy.stats, incluyendo como parámetros el valor de *F*, los grados de libertas del EMS (m), y los grados de libertas del RMS (n-m-1).

Al finalizar, imprimiremos en consola el valor del estadístico *F* y de su *p-value* asociado, para comprobar que son iguales a los resultados mostrados en el `summary` del modelo. Recuerda que deberás importar ambas librerías consideradas en los pasos previos.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import scipy.stats as st<br>
    import numpy as np<br>
    yhat = results.predict(sm.add_constant(X))<br>
    ybar = np.mean(Y)<br>
    ESS = sum((yhat - ybar)**2)<br>
    m = X.shape[1]<br>
    EMS = ESS / m<br>
    RSS = sum((Y - yhat)**2)<br>
    n = X.shape[0]<br>
    RMS = RSS / (n - m - 1)<br>
    F = EMS / RMS<br>
    pval = st.f.sf(F, m, n - m - 1)<br>
    print("F =", F)<br>
    print("p-value =", pval)
</details>

In [6]:
# Importar librerías
import scipy.stats as st
import numpy as np
# Calcular Y estimada
yhat = results.predict(sm.add_constant(X))
# Calcular el promedio de Y
ybar = np.mean(Y)
# Calcular el ESS
ESS = np.sum((yhat - ybar) ** 2)
# Calcular m
m = X.shape[1]
# Calcular EMS
EMS = ESS / m
# Calcular RSS
RSS = np.sum((Y - yhat) ** 2)
# Calcular n
n = X.shape[0]
# Calcular RMS
RMS = RSS / (n - m - 1)
# Calcular el estadístico F
F = EMS / RMS
# Calcular el p-value
pval = st.f.sf(F, m, n - m - 1)
# Imprimir el valor de F
print("Valor de F:", F)
# Imprimir el p-value
print("Valor p:", pval)


Valor de F: 68.38972178383592
Valor p: 5.4831895107283165e-120


Si todo salió bien, los resultados deben ser los mismos que los obtenidos mediante la función `OLS`. Ahora revisemos el *p-value* de un coeficiente individual, trabajemos con el nivel de alcohol. Si recuerdan, en clase indicamos que para hacer esto, necesitamos primero generar un nuevo modelo, muy similar al que ya generamos, pero para el que los datos no incluyan a la variables de interés, el nivel de alcohol en este caso.

Empecemos con eso, definamos `XNew` como `X` sin la columna de alcohol usando la función `drop`. Generemos un nuevo modelo usando la función `OLS` y estos nuevos datos, pero usando la misma variable de salida `Y`. Ajustemos el modelo, para así obtener los coeficientes estimados, usando el método `fit()`. Listo, tenemos un nuevo modelo que no incluye la variable de interés.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    XNew = X.drop('alcohol', axis = 1)<br>
    modelNew = sm.OLS(Y,sm.add_constant(XNew))<br>
    resultsNew = modelNew.fit()
</details>

In [8]:
# Genera el nuevo elemento X, sin la variable alcohol
XNew = train.drop(['calidad', 'alcohol'], axis=1)
# Define el nuevo modelo
modelNew = sm.OLS(Y, sm.add_constant(XNew))
# Ajusta el nuevo modelo
resultsNew = modelNew.fit()


Ahora calculemos el *p-value* asociado, siguiendo la ecuación mostrada en clase. Realizaremos una serie de pasos muy similares a los usados para calcular el estadístico *F* global:
1. Estimar el nuevo valor de $Y$ usando el método `predict` en los nuevos datos.
2. Calcular el nuevo valor del RSS.
3. Calcular el nuevo valor del EMS ((RSSNew - RSS)/1).
4. Obtener el nuevo valor de F (EMSNew/RMS).
5. Calcular el *p-value* asociado a dicho estadístico.
6. Ya que lo que el `summary` muestra es el valor del estadístico *t*, calculamos dicho valor como la raíz cuadrada de *F* con la función `sqrt` de numpy.

Ahora solo resta imprimir en consola cada uno de estos valores (nuevo valor de *F*, *t*, y nuevo *p-value*) y compararlos con los que muestra la tabla del `summary` unas cuantas celdas arriba de esta. Si el *p-value* del alcohol solo se lee como 0.000, usa el atributo `pvalues` para la columna `alcohol` en la variable de resultados, e imprime dicho valor en consola.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    yhatNew = resultsNew.predict(sm.add_constant(XNew))<br>
    RSSNew = sum((Y-yhatNew)**2)<br>
    EMSNew = (RSSNew - RSS) / 1<br>
    FNew = EMSNew / RMS<br>
    pvalNew = st.f.sf(FNew, 1, n-m-1)<br>
    t = np.sqrt(FNew)<br>
    print("New F =", FNew)<br>
    print("t-value =", t)<br>
    print("p-value =", pvalNew)<br>
    print("OLS's p-value =", results.pvalues.alcohol)
</details>

In [14]:
# Realiza las nuevas estimaciones de Y
yhatNew = resultsNew.predict(sm.add_constant(XNew))
# Calcula el nuevo RSS
RSSNew = sum((Y - yhatNew) ** 2)
# Calcula el nuevo EMS
EMSNew = (RSSNew - RSS) / 1
# Calcula el nuevo estadístico F
FNew = EMSNew / RMS
# Obtén el nuevo p-value
pvalNew = st.f.sf(FNew, 1, n - m - 1)
# Calcula el estadístico t como la raíz cuadrada de F
t = np.sqrt(FNew)
# Imprime el nuevo valor de F
print("Nuevo valor de F:", FNew)

# Imprime el valor de t
print("Valor de t:", t)

# Imprime el p-value
print("Nuevo p-value:", pvalNew)

# Imprime el p-value entregado por la función OLS
print("p-value de OLS:", results.pvalues.alcohol)

Nuevo valor de F: 100.0163484304423
Valor de t: 10.000817388115948
Nuevo p-value: 1.0242979671312324e-22
p-value de OLS: 1.0242979671334248e-22


Con esto, hemos comprobado que somos capaces de generar y analizar los resultados de una regresión lineal múltiple, incluso si en la práctica dejamos de calcular cada una de estas métricas de forma manual en el futuro. Pero, aún queda algo pendiente, recuerda que en un inicio separamos nuestros datos en `train` y `test`. Revisemos el ajuste del modelo en los datos de validación, calculando tanto el RSE como la $R^2$, siguiendo las ecuaciones mostradas en clase.

De nuevo, seguiremos una serie de pasos:
1. Generar `XTest` de forma similar a como se generó `X`, pero usando solo los datos de `test`.
2. Estimar el valor de $Y$ para los datos de validación.
3. Generar `YTest` de forma similar a como se generó `Y`.
4. Calcular RSSTest.
5. Calcular TSSTest, estimando el promedio de `YTest` con la función `mean` de numpy.
6. Obtener el valor de `nTest`, la cantidad de observaciones de los datos de validación (debe tener un valor distinto a `n`, pues hay más sujetos en el set de entrenamiento que en el de validación).
7. Obtener el valor de `mTest`, la cantidad de variables en los datos de validación (debe tener el mismo valor que `m`, pues usamos las mismas variables para entrenar y validar).
8. Calcular RSETest (*residual standard error* en validación).
9. Calcular R2Test (R cuadrada en validación).

Al finalizar, imprime ambas métricas en consola.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    XTest = test.drop('calidad', axis = 1)<br>
    yhatTest = results.predict(sm.add_constant(XTest))<br>
    YTest = test.calidad<br>
    RSSTest = sum((YTest-yhatTest)**2)<br>
    TSSTest = sum((YTest-np.mean(YTest))**2)<br>
    nTest = XTest.shape[0]<br>
    mTest = XTest.shape[1]<br>
    RSETest = np.sqrt(RSSTest/(n-m-1))<br>
    R2Test = 1 - RSSTest / TSSTest<br>
    print("RSE =", RSETest)<br>
    print("R^2 =", R2Test)<br>
</details>

In [15]:
# Genera el elemento XTest
XTest = test.drop('calidad', axis=1)
# Estima los valores de Y para los datos de validación
yhatTest = results.predict(sm.add_constant(XTest))
# Genera el elemento YTest
YTest = test.calidad
# Calcula el RSS de validación
RSSTest = sum((YTest - yhatTest) ** 2)
# Calcula el TSS de validación
TSSTest = sum((YTest - np.mean(YTest)) ** 2)
# Define el valor de n para los datos de prueba
nTest = XTest.shape[0]
# Define el valor de m para los datos de prueba
mTest = XTest.shape[1]
# Calcula el RSE de validación
RSETest = np.sqrt(RSSTest / (n - m - 1))
# Calcula el R^2 de validación
R2Test = 1 - (RSSTest / TSSTest)
# Imprime el RSE
print("RSE =", RSETest)
# Imprime el R^2
print("R^2 =", R2Test)


RSE = 0.3287150663189919
R^2 = 0.3023004971112384


En esta ocasión, los valores obtenidos no serán los mismos que los que se observan en `summary`, pues se está trabajando con otros datos. Sin embargo, es importante revisar que la diferencia no sea muy grande, pues eso indicaría problemas en nuestro modelo, típicamente relacionados a *overfitting*. Si estás interesado en revisar el RSE de entrenamiento, dicha métrica no se muestra en la tabla. Sin embargo, dicho valor puede calcularse de forma sencilla como la raíz cuadrada del atributo `scale` de los resultados.

Listo, hemos terminado un análisis muy completo. Estás listo para realizar tus propios análisis e interpretar sus resultados.